In [2]:
# Importing Basic Python Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import math # Importing math

# Abstract Syntax Trees
import ast

In [4]:
# Importing articles.csv file which has profiling info of all articles
articles=pd.read_csv('devopedia/devopedia/spiders/articles_152.csv')

In [5]:
articles.isnull().sum()

title         0
summary       0
QA            0
references    0
file_name     0
dtype: int64

In [6]:
articles.head()

,title,summary,QA,references,file_name
0,MQTT,MQTT is a messaging transport protocol. It ha...,"[{'question': ""What's the expansion for MQTT?""...",['http://docs.oasis-open.org/mqtt/mqtt/v3.1.1/...,MQTT_ref.csv
1,Race Condition (Software),Race condition in software is an undesirable e...,[{'question': 'Can you give one example of rac...,['https://clang.llvm.org/docs/ThreadSafetyAnal...,Race Condition (Software)_ref.csv
2,HTTP/2,HTTP/2 is an alternative to HTTP/1.x that has ...,[{'question': 'Are there any published results...,['http://httpd.apache.org/docs/2.4/mod/mod_htt...,HTTP-2_ref.csv
3,Li-Fi,"Li-Fi, or Light Fidelity, uses light as a medi...",[{'question': 'Why do we need Li-Fi when plent...,['http://www.eenewseurope.com/news/li-fi-achie...,Li-Fi_ref.csv
4,Web Annotation,"In traditional print media, it's common practi...","[{'question': 'Without Web Annotation, how do ...",['https://web.hypothes.is/blog/3-million-annot...,Web Annotation_ref.csv


There is an article on "Dotdot" which actually has No Reference links. So the filename column there is actually labelled as "No References". 
https://devopedia.org/dotdot

In [7]:
articles[articles.file_name=="No References"]

,title,summary,QA,references,file_name
75,Dotdot,is the universal language of the Internet of ...,"[{'question': ""What's the need to have Dotdot?...",[],No References


In [8]:
for index,row in articles.iterrows():
    if str(row['file_name'])=="No References":
        continue
    # text='devopedia/devopedia/spiders/'+row['file_name'] for reading file
    text=pd.read_csv('devopedia/devopedia/spiders/'+row['file_name'])
    
    text=text.replace(np.nan,"",regex=True) # replacing 'nan' with blanks
    
    # Preparing Dictionary of Columns for our Clean Data Frames for each article
    cleaned_df_columns={'title', 'reference_link', 'type', 'text'}
    
    # Assigning the columns to the cleaned_df DataFrame
    cleaned_df=pd.DataFrame(columns=cleaned_df_columns)
    
    # Iterating over each reference link using iterrows again
    # Here will use counters named i,r instead of index,rows
    # Extracting the text in between 'Abstract' and 'References' inside pdf
    
    for i,r in text.iterrows():
        cleaned_text=r['text']
        if  cleaned_text=="403 Forbidden\n403 Forbidden\nnginx":
            cleaned_text=""
        if(r['type'].find("pdf")!=-1):
            abstract_index=r['text'].find("Abstract")
            if  abstract_index==-1:
                abstract_index=0
            refer_index=r['text'].rfind("References")
            cleaned_text=r['text'][abstract_index:refer_index]
        
        # Initializing dict to contain cleaned_df cols as the keys
        row_dict={}
        
        # Assigning values from row to the cleaned_df columns
        row_dict['title']=r['title']
        row_dict['reference_link']=r['reference_link']
        row_dict['type']=r['type']
        row_dict['text']=cleaned_text
        
        # Appending row_dict to cleaned_df
        cleaned_df=cleaned_df.append(row_dict,ignore_index=True)
    
    # Writing cleaned_df for each article to_csv 
    # in the'devopedia/devopedia/spiders/' path
    cleaned_df.to_csv('devopedia/devopedia/spiders/'+row['file_name'][:-4]+"_cleaned.csv")